In [2]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
import gzip
import json

In [4]:
with gzip.open('NCSU-DigIC-GraphData-2023-07-25/cells.json.gz','rb') as f:
    cells = json.loads(f.read().decode('utf-8'))

FileNotFoundError: [Errno 2] No such file or directory: 'NCSU-DigIC-GraphData-2023-07-25/cells.json.gz'

In [3]:
data = pd.read_csv('eda/preprocessed_data.csv')
data

,name,id,xloc,yloc,cell,orient,routing_demand,congestion,capacity,GRC_Index,pin_count,pin_total
0,clk_gate_out_reg/latch,0,41984,44544,23,0,20.0,-8.0,28.0,"(29, 27)",4,12
1,clk_gate_out_reg_0/latch,1,41984,47616,23,6,23.0,-5.0,28.0,"(31, 27)",4,8
2,clk_gate_out_reg_1/latch,2,44160,44544,23,0,23.0,-10.0,33.0,"(29, 28)",4,10
3,clk_gate_out_reg_2/latch,3,44160,47616,23,0,22.0,-11.0,33.0,"(31, 28)",4,8
4,clk_gate_out_reg_3/latch,4,46336,47616,23,0,21.0,-2.0,23.0,"(31, 29)",4,4
...,...,...,...,...,...,...,...,...,...,...,...,...
3947,U4123,3947,21888,53760,42,4,31.0,0.0,31.0,"(35, 13)",3,12
3948,U4125,3948,33664,66048,42,0,30.0,0.0,30.0,"(43, 21)",3,16
3949,U4128,3949,23296,66048,34,0,27.0,-2.0,29.0,"(43, 14)",2,17
3950,ZCTSBUF_205_132,3950,40576,44544,11,0,28.0,-5.0,33.0,"(29, 26)",2,12


In [4]:
individual_pins = []
cell_widths = []
cell_heights = []
in_connections = []
out_connections = []
for k in range(data.shape[0]):
    cell_type = data.iloc[k]['cell']
    cell_pins = len(cells[cell_type]['terms'])
    cell_in_connections = 0
    cell_out_connections = 0
    for term in cells[cell_type]['terms']:
        if term['dir'] == 0:
            cell_in_connections += 1
        if term['dir'] == 1:
            cell_out_connections += 1
    cell_width = cells[cell_type]['width']
    cell_height = cells[cell_type]['height']
    individual_pins.append(cell_pins)
    cell_widths.append(cell_width)
    cell_heights.append(cell_height)
    in_connections.append(cell_in_connections)
    out_connections.append(cell_out_connections)

data['individual_pins'] = individual_pins
data['width'] = cell_widths
data['height'] = cell_heights
data['in_connections'] = in_connections
data['out_connections'] = out_connections
data

,name,id,xloc,yloc,cell,orient,routing_demand,congestion,capacity,GRC_Index,pin_count,pin_total,individual_pins,width,height,in_connections,out_connections
0,clk_gate_out_reg/latch,0,41984,44544,23,0,20.0,-8.0,28.0,"(29, 27)",4,12,4,2176,1536,3,1
1,clk_gate_out_reg_0/latch,1,41984,47616,23,6,23.0,-5.0,28.0,"(31, 27)",4,8,4,2176,1536,3,1
2,clk_gate_out_reg_1/latch,2,44160,44544,23,0,23.0,-10.0,33.0,"(29, 28)",4,10,4,2176,1536,3,1
3,clk_gate_out_reg_2/latch,3,44160,47616,23,0,22.0,-11.0,33.0,"(31, 28)",4,8,4,2176,1536,3,1
4,clk_gate_out_reg_3/latch,4,46336,47616,23,0,21.0,-2.0,23.0,"(31, 29)",4,4,4,2176,1536,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3947,U4123,3947,21888,53760,42,4,31.0,0.0,31.0,"(35, 13)",3,12,3,512,1536,2,1
3948,U4125,3948,33664,66048,42,0,30.0,0.0,30.0,"(43, 21)",3,16,3,512,1536,2,1
3949,U4128,3949,23296,66048,34,0,27.0,-2.0,29.0,"(43, 14)",2,17,2,384,1536,1,1
3950,ZCTSBUF_205_132,3950,40576,44544,11,0,28.0,-5.0,33.0,"(29, 26)",2,12,2,640,1536,1,1


In [5]:
X = data[['xloc', 'yloc', 'pin_count', 'width', 'in_connections', 'out_connections']]
y = data['routing_demand']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [6]:
model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

Error: Canceled future for execute_request message before replies were done